<a href="https://colab.research.google.com/github/Skquark/structured-prompt-generator/blob/main/Enhanced_Stable_Diffusion_k_lms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [STABLE-K-4D](https://github.com/DamascusGit/stable-diffusion) - **Enhanced Stable Diffusion**

Jupyter/Colab implementation of [Stable Diffusion](https://github.com/CompVis/stable-diffusion) and Katherine Crowson's [k-diffusion wrapper](https://github.com/crowsonkb/k-diffusion) in order to use the k_lms sampler  by [karan4d](https://twitter.com/karan4d) aka [DamascusGit](https://github.com/DamascusGit). The notebook utilizes CPU draw manual seeding with k_lms and has a weird quantize.py adjustment in an attempt to best replicate the results from Simulacra/Dreambot using the stable-diffusion weights.

implemented with the help of [Katherine Crowson](https://https://twitter.com/rivershavewings), palp, wbrown, and litexev. shoutout SimulacraBot/DreamBot and their team, as well as the good folks over at EleutherAI + StabilityAI.

---
Mods and Flow Enhancements by [**Skquark**](https://www.Skquark.com)

You can use the updated implementation of [**Enhanced Stable Diffusion**](https://colab.research.google.com/github/Skquark/structured-prompt-generator/blob/main/Enhanced_Stable_Diffusion_with_diffusers.ipynb) instead..

Try our other useful notebooks [Structured Prompt Generator](https://colab.research.google.com/github/Skquark/structured-prompt-generator/blob/main/Structured_Prompt_Generator.ipynb) and [Enhanced DiscoArt](https://colab.research.google.com/github/Skquark/structured-prompt-generator/blob/main/DiscoArt_%5B_w_Batch_Prompts_%26_GPT_3_Generator%5D.ipynb)

In [ ]:
#@title Check GPU Status (A100 > G100 > V100 > P100 > T4 > K8)
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
    #!nvidia-smi
    nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
else:
    #!nvidia-smi -i 0 -e 0
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
    nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_ecc_note)

In [ ]:
#@title ## Mount your Google Drive (Colab only)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

#@title ## Choose Installation Options
#@markdown You can save the project to your Google Drive, which takes about 8GB, for faster startup. Also saves your image outputs to your GDrive path (or elsewhere) for convienience. If you already have sd-v1-3-full-ema.ckpt include path to file.
install_type = "Google Colab" #@param ['Google Colab', 'Jupyter Notebook']
copy_to_GDrive = False #@param {'type': 'boolean'}
image_output = '/content/drive/MyDrive/AI/Stable_Diffusion/images_out' #@param {'type': 'string'}
download_sd_model = True #@param {'type': 'boolean'}
download_HuggingFace_model  = False #@param {'type': 'boolean'}
HuggingFace_api_key = '' #@param {'type': 'string'}
#save_models_to_google_drive = True #@param {'type': 'boolean'}
ckpt_file = '' #@param {'type': 'string'}
#ckpt_file = '/content/drive/MyDrive/StableRun/stable-diffusion/models/ldm/stable-diffusion-v1/model.ckpt'
root_dir = '/content/'
if install_type is 'Google Colab':
  root_dir = '/content/'
elif install_type is 'Jupyter Notebook':
  root_dir = '/workspace/'
stable_dir = root_dir
if copy_to_GDrive:
  stable_dir += 'drive/MyDrive/AI/'
stable_dir += 'Stable_Diffusion'
if not os.path.exists(stable_dir):
  os.makedirs(stable_dir)
if not os.path.exists(image_output):
  os.makedirs(image_output)
os.chdir(stable_dir)

In [ ]:
#@title ## Install Conda Colab Libraries
!pip install -q condacolab
import condacolab
condacolab.install()

!conda --version
#If !conda --version returns no results, install conda with :
#!pip install -q condacolab
#import condacolab
#condacolab.install()

!ls

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:29
🔁 Restarting kernel...
conda 4.9.2
condacolab_install.log


In [ ]:
import os
#@title # First-time Setup and SD Model Download
os.chdir(stable_dir)
!git clone https://github.com/DamascusGit/stable-diffusion
import os
import shutil

model_ckpt = stable_dir + '/stable-diffusion/models/ldm/stable-diffusion-v1/model.ckpt'
if not os.path.exists(f'{stable_dir}/stable-diffusion/models/ldm/stable-diffusion-v1/'):
  os.mkdir(f'{stable_dir}/stable-diffusion/models/ldm/stable-diffusion-v1/')
if download_sd_model:
  if not os.path.exists(model_ckpt):
    !wget -q --show-progress --no-cache --backups=1 'https://drinkordiecdn.lol/sd-v1-3-full-ema.ckpt'
    shutil.move(stable_dir + '/sd-v1-3-full-ema.ckpt', model_ckpt)
else:
  if bool(ckpt_file):
    if os.path.exists(ckpt_file):
      if not os.path.exists(model_ckpt):
        shutil.copy(ckpt_file, model_ckpt)
    else: print("Couldn't find your sd checkpoint model")
  elif not os.path.exists(model_ckpt):
    print("Put in the path to your sd-v1-3-full-ema.ckpt file, or click download leaked model")
#!ln -s /content/sd-v1-3-full-ema.ckpt models/ldm/stable-diffusion-v1/model.ckpt
!ls -l {stable_dir}/stable-diffusion/models/ldm/stable-diffusion-v1
os.chdir(f'{stable_dir}/stable-diffusion')

In [ ]:
#@title # Redirect to /stable-diffusion path if do a Restart Runtime
os.chdir(f'{stable_dir}/stable-diffusion')
# use this for Jupyter
#%cd /workspace/StableRun/stable-diffusion
#%cd /content/StableRun/stable-diffusion
# use this for Colab
#%cd /content/drive/MyDrive/StableRun/stable-diffusion

In [ ]:
#@title # Install Environment and Dependency Code
os.chdir(f'{stable_dir}/stable-diffusion')
!conda env create -f environment.yaml
!conda activate ldm

!conda install pytorch torchvision -c pytorch -y
!pip install transformers==4.19.2
!pip install -e .
!conda install omegaconf
!pip install omegaconf
!pip install einops
!pip install pytorch_lightning
#!pip install git+https://github.com/Compvis/latent-diffusion.git
#!conda install pytorch==1.9.0 torchvision==0.10.1 -c pytorch -c conda-forge
#!pip install ldm
#!pip install git+https://github.com/CompVis/latent-diffusion/tree/main/models/ldm.git
#!pip install dlib
#!pip install pillow==4.0.0
!pip install scikit-image
!conda install Pillow==6.1 #USE THIS WITH COLAB
!pip install opencv-python
!pip install ai-tools
!pip install cognitive-face
!pip install zprint
#!pip install setuptools==59.5.0
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install git+https://github.com/openai/CLIP.git
!pip install kornia

In [ ]:
#@title ## Install k-diffusion requirements
#@markdown Run to make sure you install requirements appropriate for Jupyter or Colab.
%pip install lpips
%pip install keras
%pip install git+https://github.com/crowsonkb/k-diffusion/
# Can't find requirements file.. Is it mistake?
#%pip install -r /content/drive/MyDrive/requirements.txt #IF USING COLAB
#%pip install -r /workspace/StableRun/requirements.txt #IF USING JUPYTR

In [ ]:
#@markdown ### To use **Real-ESRGAN AI Upscaling**, run this first
# # Clone Real-ESRGAN and enter the Real-ESRGAN
%cd {root_dir}
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
%cd ..

## Parameter Cheatsheet

```
usage: txt2img.py [-h] [--prompt [PROMPT]] [--outdir [OUTDIR]] [--skip_grid] [--skip_save] [--ddim_steps DDIM_STEPS] [--plms] [--laion400m] [--fixed_code] [--ddim_eta DDIM_ETA] [--n_iter N_ITER] [--H H] [--W W] [--C C] [--f F] [--n_samples N_SAMPLES] [--n_rows N_ROWS]
                   [--scale SCALE] [--from-file FROM_FILE] [--config CONFIG] [--ckpt CKPT] [--seed SEED] [--precision {full,autocast}]

 optional arguments:
   -h, --help            show this help message and exit
   --prompt [PROMPT]     the prompt to render
   --outdir [OUTDIR]     dir to write results to
   --skip_grid           do not save a grid, only individual samples. Helpful when evaluating lots of samples
   --skip_save           do not save individual samples. For speed measurements.
   --ddim_steps DDIM_STEPS
                         number of ddim sampling steps
   --plms                use plms sampling
   --laion400m           uses the LAION400M model
   --fixed_code          if enabled, uses the same starting code across samples
   --ddim_eta DDIM_ETA   ddim eta (eta=0.0 corresponds to deterministic sampling
   --n_iter N_ITER       sample this often
   --H H                 image height, in pixel space (multiple of 64)
   --W W                 image width, in pixel space (multiple of 64)
   --C C                 latent channels
   --f F                 downsampling factor
   --n_samples N_SAMPLES
                         how many samples to produce for each given prompt. A.k.a. batch size
   --n_rows N_ROWS       rows in the grid (default: n_samples)
   --scale SCALE         unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))
   --from-file FROM_FILE
                         if specified, load prompts from this file
   --config CONFIG       path to config which constructs model
   --ckpt CKPT           path to checkpoint of model
   --seed SEED           the seed (for reproducible sampling)
   --precision           {full,autocast} evaluate at this precision


---
# Set your Stability.ai Parameters

In [ ]:
#prompt = "" #@param {'type': 'string'}
n_samples = 1 #@param {'type': 'integer'}
n_iterations = 1 #@param {'type': 'integer'}
ddim_steps  = 50 #@param {'type': 'integer'}
ddim_eta = 0.0 #@param {'type': 'number'}
scale = 7 #@param {'type': 'integer'}
seed = 22276 #@param {'type': 'integer'}
width = 512 #@param {'type': 'integer'}
height = 512 #@param {'type': 'integer'}
lms_sampling = True #@param {'type': 'boolean'}
plms_sampling = False #@param {'type': 'boolean'}
init_image = None #@param {'type': 'string'}
init_image_strength = 0.4 #@param {'type': 'number'}
#@markdown ---
#@markdown If you want to apply upscaling, initiallize ESRGAN library first
apply_ESRGAN_upscale = False #@param {type:"boolean"}
enlarge_scale = 2 #@param {type:'slider', min:1, max:4, step:0.5}
#@markdown ---
file_allowSpace = False #@param {type:"boolean"}
file_prefix = 'sd-' #@param {type:"string"}
save_image_metadata = False #@param {type:"boolean"}
meta_ArtistName = "Alan Bedian" #@param {type:"string"}
meta_Copyright = "Skquark, Inc." #@param {type:"string"}
params = ''
if init_image:
  params += ' --init_img ' + init_image + ' --strength ' + init_image_strength
params += f' --n_samples {n_samples}'
params += f' --n_iter {n_iterations}'
params += f' --ddim_steps {ddim_steps}'
params += f' --W {width} --H {height}'
params += f' --scale {scale}'
params += f' --seed {seed}'
if ddim_eta != 0.0:
  params += f' --ddim_eta {ddim_eta}'
if plms_sampling:
  params += ' --plms'
print('Params:' + params)

Params: --n_samples 1 --n_iter 1 --ddim_steps 50 --W 512 --H 512 --scale 7 --seed 22276


### Put one or many text prompts in the list, then run:

In [ ]:
prompts = [
    "Intergalactic stargate portal in the middle of a forest, with animals and birds all around it",
    "Intergalactic stargate portal in the middle of a city, with people all around it by Frederic Church and Larry Elmore, style of stonepunk",
]

In [ ]:
#@title ## **> Run Stable Diffusion** on prompts
import string, shutil, random, gc
from collections import ChainMap
from PIL import Image
from PIL.PngImagePlugin import PngInfo
#import torch

os.chdir(f'{stable_dir}/stable-diffusion')
def format_filename(s):
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    filename = ''.join(c for c in s if c in valid_chars)
    if file_allowSpace: filename = filename.replace(' ','_')
    return filename[:248]

for p in prompts:
  try:
    if lms_sampling:
      !python scripts/txt2img_k.py --prompt "{p}"{params}
    else: # Ignore Invalid character error, still works, their bug
      !python scripts/txt2img.py --prompt "{p}"{params}
  except RuntimeError as e:
    if 'out of memory' in str(e):
      print(f"\u001b[31m\u001b[1m\u001b[4mCRITICAL ERROR\u001b[0m: GPU ran out of memory! Flushing memory to save session...")
      pass
  finally:
    gc.collect()
    #torch.cuda.empty_cache()
  txt2img_output = f'{stable_dir}/stable-diffusion/outputs/txt2img-samples'
  if not os.path.exists(txt2img_output):
    os.makedirs(txt2img_output)
    print("Couldn't find output folder " + txt2img_output)
  else:
    filenames = os.listdir(txt2img_output)
    filename = format_filename(p)
    idx = 0
    for fname in filenames:
      if fname.endswith('png'):
        fpath = os.path.join(txt2img_output, fname)
        if apply_ESRGAN_upscale:
          os.chdir(f'{root_dir}Real-ESRGAN')
          upload_folder = 'upload'
          result_folder = 'results'     
          if os.path.isdir(upload_folder):
              shutil.rmtree(upload_folder)
          if os.path.isdir(result_folder):
              shutil.rmtree(result_folder)
          os.mkdir(upload_folder)
          os.mkdir(result_folder)
          dst_path = os.path.join(f'{root_dir}Real-ESRGAN/{upload_folder}', fname)
          #print(f'Moving {fpath} to {dst_path}')
          shutil.move(fpath, dst_path)
          # Ignore Invalid character error, still works, their bug
          !python inference_realesrgan.py -n RealESRGAN_x4plus -i upload --outscale {enlarge_scale}
          out_file = fname.rpartition('.')[0] + '_out.png'
          #print(f'move {root_dir}Real-ESRGAN/{result_folder}/{out_file} to {fpath}')
          shutil.move(f'{root_dir}Real-ESRGAN/{result_folder}/{out_file}', fpath)
          # !python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input upload --netscale 4 --outscale 3.5 --half --face_enhance
          os.chdir(f'{stable_dir}/stable-diffusion')
        if save_image_metadata:
          img = Image.open(fpath)
          metadata = PngInfo()
          metadata.add_text("artist", meta_ArtistName)
          metadata.add_text("copyright", meta_Copyright)
          metadata.add_text("software", "Stable Diffusion 1.3" + f", upscaled {enlarge_scale}x with ESRGAN" if apply_ESRGAN_upscale else "")
          metadata.add_text("title", p)
          #metadata.add_text("prompt", p)
          #metadata.add_text("config", str(d.tags))
          img.save(fpath, pnginfo=metadata)
        new_file = f'{file_prefix}{filename}'
        if os.path.isfile(os.path.join(image_output, f'{new_file}-{idx}.png')):
          new_file += '-' + random.choice(string.ascii_letters) + random.choice(string.ascii_letters)
        new_file += f'-{idx}.png'
        shutil.move(fpath, os.path.join(image_output, new_file))
        idx += 1


In [ ]:
#@title ### Out of memory? Try running this...

#import torch
#torch.cuda.empty_cache()

#taskkill /F /PID <your PID here>

#CUDA torch not enabled? try this:
!pip uninstall torch
!pip install torch --pre --extra-index-url https://download.pytorch.org/whl/nightly/cu116

# Manually Run txt2img scripts

In [ ]:
#USING COLAB PRO HIGH-RAM (CHANGE SEED EACH TIME):

#txt2img no k_lms sampling
#!python scripts/txt2img.py --prompt "photorealistic award-winning photograph of a dog on a unicycle " --n_samples 1 --n_iter 1 --ddim_steps 50 --H 640 --scale 7 --seed 4314123 --plms

#txt2img with k_lms sampling
!python scripts/txt2img_k.py --prompt "photorealistic award-winning photograph of a dog on a unicycle " --n_samples 1 --n_iter 1 --ddim_steps 50 --H 640 --scale 7 --seed 1134143 

Global seed set to 1134143
Loading model from models/ldm/stable-diffusion-v1/model.ckpt
Global Step: 440000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
^C


In [ ]:
#USING 1xA6000 OR EQUIVALENT (CHANGE SEED EACH TIME):

#txt2img no k_lms sampling
!python scripts/txt2img.py --prompt "photorealistic award-winning photograph of a dog on a unicycle " --n_samples 3 --n_iter 1 --ddim_steps 50 --H 832 --W 576 --scale 10 --seed 9235342 --plms
#txt2img with k_lms sampling
!python scripts/txt2img_k.py --prompt "photorealistic award-winning photograph of a dog on a unicycle " --n_samples 3 --n_iter 1 --ddim_steps 50 --H 832 --W 576 --scale 10 --seed 9235342

In [ ]:
#image2image script

!python scripts/img2img.py --prompt "photorealistic award-wining photograph of a dog on a unicycle" --init-img /imagepath/image.png --strength 0.4 --seed 1574552011 --scale 7 --n_samples 1 --n_iter 1

# DEBUG CODE - IGNORE

In [ ]:

!mkdir -p models/ldm/stable-diffusion-v1/

%cd models/ldm/stable-diffusion-v1/

!ln -s /content/drive/My Drive/stable-diffusion-v-1-3/sd-v1-3-full-ema.ckpt /content/drive/My\ Drive/Stable\ Run/stable-diffusion/models/ldm/stable-diffusion-v1/

#OUTDATED:
#REPLACE ldm.py in usr/local/lib/python3.7/site-packages/ldm.py WITH the one in /content/drive/My Drive/Stable\ Run/ldm.py
#REPLACE microsoft_demo.py in /usr/local/lib/python3.7/site-packages/ai_tools/microsoft_demo.py WITH the one in /content/drive/My Drive/Stable\ Run/microsoft_demo.py

In [ ]:
!pip freeze

In [ ]:
!nvidia-smi

Fri Aug 19 06:52:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------